In [ ]:
# imports
from subprocess import check_output
import pandas as pd
import plotly.express as px
import plotly
import re
import nbformat
import time

In [ ]:
# run code for each p value repeat times from 0.1 to 0.9
scores = []
repeat = 20

for i in range(10):
    count = 0
    p = 0.1 * i
    for x in range(repeat):
        command = 'python main.py -d 101 -a a_star -p {}'.format(p)
        out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
        if 'No Path Found' in out:
            continue
        else:
            count += 1
    count /= repeat
    scores.append(count)

print(scores)

In [ ]:
# conver the scores list to a dict
scores_dict = dict(zip([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], scores))
scores_df = pd.DataFrame.from_dict(scores_dict, orient='index').reset_index()
scores_df.columns = ["probability", "solvability"]
# plot the scores array

fig = px.line(scores_df, x="probability", y="solvability", title='Plot Density vs Solvability')
fig.show()

plotly.io.write_image(fig, file="./graphs/question_4.png", scale=5)

In [ ]:
# measure heuristic efficiency
times_df = pd.DataFrame(columns = ['Time', 'Heuristic'])

for x in range(100):
    command = 'python main.py -d 500 -a a_star -m all -p 0.4'
    out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
    formatted = out.rstrip("]\\n'").lstrip("b'[").split(', ')
    list(map(float, formatted))

    times_df = times_df.append([{'Time': formatted[0], 'Heuristic': 'Chebyshev'}, {'Time': formatted[1], 'Heuristic': 'Manhattan'}, {'Time': formatted[2], 'Heuristic': 'Euclidian'}], ignore_index=True)

    # print(formatted)


print(times_df)

In [ ]:
# plot the box plots
fig = px.box(times_df, x="Heuristic", y="Time", title="Heuristics vs Time")
fig.show()
plotly.io.write_image(fig, file="./graphs/question_5.jpg", scale=5)

In [ ]:
question_six_df = pd.DataFrame(columns = ['Density', 'Trajectory', 'Discovered Shortest Path', 'Final Shortest Path', 'Processed Cells', 'Trial'])

for i in range(100):
    for x in range(12):
        p = 0.03*x
        command = 'python main.py -d 101 -a repeated_a_star -m all -p ' + str(p)

        while True:
            out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
            trajectory = int(re.search(r"Trajectory Length: \d+", out).group().lstrip("Trajectory Length: "))
            shortest_path = int(re.search(r"Shortest Path: \d+", out).group().lstrip("Shortest Path: "))
            full_world_shortest_path = int(re.search(r"Full Gridworld Path: \d+", out).group().lstrip("Full Gridworld Path: "))
            processed_cells = int(re.search(r"Processed \d+", out).group().lstrip("Processed "))

            if shortest_path > 0:
                break
        
        question_six_df = question_six_df.append([{'Density': p, 'Trajectory': trajectory, 'Discovered Shortest Path': shortest_path, 'Final Shortest Path': full_world_shortest_path, 'Processed Cells': processed_cells, 'Trial': i+1}])
# print(question_six_df)

In [ ]:
grouped_df = question_six_df.groupby('Density')
avg_df = grouped_df.mean()
avg_df = avg_df.reset_index()

In [ ]:
# do some calculations
avg_df['Trajectory / Shortest Path (Disc.)'] = round(avg_df['Trajectory'] / avg_df['Discovered Shortest Path'], 3)
avg_df['Shortest Path (Disc.) / Shortest Path (Final)'] = round(avg_df['Discovered Shortest Path'] / avg_df['Final Shortest Path'], 3)

In [ ]:

fig1 = px.line(avg_df, x="Density", y="Trajectory", title='Density vs Average Trajectory Length', text='Trajectory')
fig1.update_xaxes(dtick=0.03)
fig1.update_traces(textposition="bottom right")
fig1.show()

plotly.io.write_image(fig1, file="./graphs/question_6_1.png", scale=5)

In [ ]:
fig2 = px.line(avg_df, x="Density", y="Trajectory / Shortest Path (Disc.)", title='Density vs Average Shortest Path Lengths', text='Trajectory / Shortest Path (Disc.)')
fig2.update_xaxes(dtick=0.03)
fig2.update_traces(textposition="bottom right")
fig2.show()

plotly.io.write_image(fig2, file="./graphs/question_6_2.png", scale=5)

In [ ]:
fig3 = px.line(avg_df, x="Density", y="Shortest Path (Disc.) / Shortest Path (Final)", title='Density vs Average Shortest Path Lengths', text='Shortest Path (Disc.) / Shortest Path (Final)')
fig3.update_xaxes(dtick=0.03)
fig3.update_traces(textposition="bottom right")
fig3.show()

plotly.io.write_image(fig3, file="./graphs/question_6_3.png", scale=5)

In [ ]:
fig4 = px.line(avg_df, x="Density", y="Processed Cells", title='Density vs Processed Cells', text='Processed Cells')
fig4.update_xaxes(dtick=0.03)
fig4.update_traces(textposition="bottom right")
fig4.show()

plotly.io.write_image(fig3, file="./graphs/question_6_4.png", scale=5)